# Capstone Project - **Concierge Agent** Category 

## *CoMailAgent* – Automated Resume Tailoring & Cold Email Job Outreach Agent 
- by Supriya & Sanya

Find below the code & instructions for this project

### Step 1: Configuring Google API Keys
This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

### Step 2: Import ADK components

Now, importing the specific components we will need from the Agent Development Kit and the Generative AI library. This keeps the code organized and ensures we have access to the necessary building blocks.

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
FILE_ID = user_secrets.get_secret("FILE_ID")
SPREADSHEET_ID = user_secrets.get_secret("SPREADSHEET_ID")
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )


✅ Setup and authentication complete.


In [14]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=1b0e4a2b4b7aa7094b766a5b5bf2edde6f6ef370d5da892d2531a114eb08e05d
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [32]:
import pandas as pd
import requests 
import os
from fpdf import FPDF
import io

In [8]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Step 3: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [9]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

1. root_agent - Uses the 
1. get_task_items() tool
    * Goes through the db and returns a dictionary with 
2. cover_letter_agent - for writing a custom cover letter
    * **Extracts keywords and role expectations** from the job description.
    * ~~**Tailors the resume** to highlight relevant skills and achievements.~~
    * **Generates a professional, customized cover letter** for each job.
3. email_agent - for drafting a recruiter outreach email
    * **Creates a personalized cold outreach email** addressed to the recruiter or hiring manager.
    * **Packages attachments** (tailored resume + cover letter).
    * **Sends the email automatically**, or optionally requests user approval before sending.

In [5]:
# print(os.path.join(os.getcwd(), 'filename.pdf'))

/kaggle/working/filename.pdf


In [16]:
# # testing
# google_sheet_url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=xlsx"
# job_dataset = pd.read_excel(google_sheet_url, sheet_name="Sheet1").to_dict(orient='list')
# print(job_dataset)
# print(job_dataset[0:1].to_dict(orient='list'))

{'job_id': [1], 'job_description': ['Description 1'], 'recruiter_name': ['rachael x'], 'recruiter_email': ['rachael@reatil.com']}


In [21]:
# # testing
# format = 'txt'
# url = f"https://docs.google.com/document/d/{FILE_ID}/export?format=txt"
# # The export URL format for Google Docs
# # url = f"docs.google.com{format}&id={FILE_ID}"
    
# try:
#     response = requests.get(url)
#     response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
#     print(response.text)
# except requests.exceptions.RequestException as e:
#     print(f"Error during download: {e}")
#     #return None

﻿Dear Recruitment Team,
I am writing to express my interest in data-focused opportunities—whether as a Data Engineer or Data Analyst—at your organization. As a data enthusiast with a solid foundation in data science and analytics, I’ve recently deepened my expertise by completing rigorous data engineering programs and applying those skills in hands-on capstone and pro bono projects designed to solve real-world challenges.
My earlier experience as a Data Scientist and Data Engineer at DeepMiner gave me a strong grounding in building ETL pipelines, developing NLP models, and creating analytics dashboards. I’ve since expanded my capabilities through continuous learning and project-based work aligned with modern data engineering and analytics practices.
Recent highlights include:
* Developing end-to-end ELT pipelines using Python, SQL, Docker, and AWS/GCP, reducing processing latency and simulating production-grade workflows.
* Designing secure cloud-native data pipelines with Google Cloud

In [24]:
# Tool that gets the job data
def get_job_data_method() -> dict[list]:
    """Downloads the data from google sheets which contains information about the jobs and recruiter.

        Returns:
            Dictionary with status and job_data dictionary.
            Success: {"status": "success", "job_data": {'job_id': [1, 2, 3], 
                'job_description':["Description 1", "Description 2", "Description 3"], 
                'recruiter_name': ['racheal x', 'ross y', 'joey z'], 
                'recruiter_email': ['rachael@retail.com', 'ross@museum.com', 'joey@restaurant.com']}}
            Error: {"status": "error", "error_message": "Could not get job data from google sheets"}
        """
    google_sheet_url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=xlsx"
    try:
        # get data from google sheets
        job_data = pd.read_excel(google_sheet_url, sheet_name="Sheet1")[0:1].to_dict(orient='list')
        if job_data:
            return {'status': 'success', 'job_data':job_data}
        else:
            return {'status': 'error', 'job_data':f"Could not retrive job data as no data available in the db"}
    except Exception as e:
        return {'status': 'error', 'job_data':f"Could not retrive job data due to the following error: {e}"}


In [27]:
# Tool that gets the cv
def get_cover_letter_method() -> str:
    """Downloads the cover letter from the url.

        Returns:
            String with the Cover Letter.
            Success: {"status": "success", "cover_letter": "The full Cover Letter Content"}
            Error: {"status": "error", "error_message": "Could not get the cover letter from google docs"}
        """
    google_doc_url = f"https://docs.google.com/document/d/{FILE_ID}/export?format=txt"
    try:
        # get data from google sheets
        response = requests.get(google_doc_url)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)job_data = pd.read_excel(google_sheet_url, sheet_name="Sheet1")[0:1].to_dict(orient='list')
        if response.text:
            return {'status': 'success', 'cover_letter':response.text}
        else:
            return {'status': 'error', 'job_data':f"Could not retrive the cover letter from google docs"}
    except requests.exceptions.RequestException as e:
        return {'status': 'error', 'job_data':f"Error during download: {e}"}


In [ ]:
# # Cover Letter Writer Agent : Writes the cover letter

# cv_writer_agent = Agent(
#     name="CvWriterAgent",
#     model=Gemini(
#         model="gemini-2.5-flash-lite",
#         retry_options=retry_config
#     ),
#     # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
#     instruction="""Following this outline strictly: {blog_outline}
#     Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
#     tools = [get_cover_letter_method],  # The result of this agent will be stored with this key.
# )

# print("✅ writer_agent created.")

In [17]:
def generate_pdf(text_content: str):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Split text into lines to handle multiline content
    lines = text_content.split('\n')
    for line in lines:
        pdf.cell(200, 10, txt=line, ln=True, align='L') # ln=True moves to the next line

    # Save the PDF to a byte buffer
    pdf_byte_buffer = io.BytesIO()
    pdf.output(pdf_byte_buffer)
    pdf_byte_buffer.seek(0) # Rewind the buffer to the beginning
    return pdf_byte_buffer.getvalue()

In [20]:
def save_text_as_pdf_method(content_to_save: str, filename: str = "cover_letter.pdf") -> dict[str, str]:
    """
    Generates a PDF from the given text content and saves it locally.

    Args:
        content_to_save: The string content to be put into the PDF.
        filename: The desired filename for the saved PDF artifact (e.g., "summary.pdf").
                  It should end with '.pdf'.

    Returns:
        A dictionary indicating the status of the operation and the file name or an error message.
    """
    print(f"--- Tool called: save_text_as_pdf, attempting to save as '{filename}' ---")

    if not content_to_save:
        return {"status": "error", "message": "No content provided to save as PDF."}

    try:
        if not filename.lower().endswith(".pdf"):
            filename += ".pdf"
            print(f"--- Info: Appended .pdf to filename. New filename: '{filename}' ---")

        print(f"--- Generating PDF bytes for: '{content_to_save[:100]}...' ---")
        pdf_bytes = generate_pdf(content_to_save)
        print(f"--- PDF bytes generated successfully (Size: {len(pdf_bytes)} bytes) ---")

        with open(filename, "wb") as f:
            f.write(pdf_bytes)
            print("Saved PDF locally.")
        return { "status": "success", "message": f"file {filename} saved locally at {os.path.join(os.getcwd(), filename)}" }
    except Exception as e:
        return { "status": "error", "error": str(e) }

In [35]:
# Root Agent
comail_agent = LlmAgent(
    name="comail_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a professional and highly intelligent job application assistant. You must strictly follow these steps and use the available tools to send custom cover letters for different jobs via email.

  Do the following in order and follow the instructions strictly

   1. Get Job Data: Use the get_job_data_method tool to get the job_data that contains information about the job id, job_description, recruiters full name which includes the first and last name and recruiters email address .
   2. Get Cover Letter: Use the get_cover_letter_method tool to get the cover letter
   3. Generate Cover Letter (CRITICAL): You are strictly prohibited from making up information. Using the job_data from Step 1, the cover letter from Step 2 write an updated cover letter based on job description. Do not makeup any information. Write the content of the cover letter keeping in mind it needs to be converted to PDF and therefore must have the right indentation and formatting.
   4. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   5. Save the cover letter text as PDF: Use the tool save_text_as_pdf_method() which accepts string data and saves the cover letter text received as PDF and then returns the status and the location of the file
   6. Generate email content: You need to write a professional letter to the recruiter for the job description and mention that the cover letter is attached
   7. Your final result should print the status of each tool and the email and then mention where the cover letter is saved
    """,
    tools=[
        get_job_data_method,
        get_cover_letter_method,
        save_text_as_pdf_method,  # Using another agent as a tool!
    ],
)

print("✅ CoMail agent created")


✅ CoMail agent created


In [40]:
# Test the currency agent
comail_runner = InMemoryRunner(agent=comail_agent)
_ = await comail_runner.run_debug(
    "Generate the email for the latest jobs"
)
print('agent run complete')


 ### Created new session: debug_session_id

User > Generate the email for the latest jobs


agent run complete


In [38]:
!pwd
!ls -lah /kaggle/working

/kaggle/working
total 12K
drwxr-xr-x 3 root root 4.0K Dec  1 03:41 .
drwxr-xr-x 5 root root 4.0K Dec  1 03:40 ..
drwxr-xr-x 2 root root 4.0K Dec  1 03:41 .virtual_documents


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session